In [1]:
import yaml
import json
import pandas as pd

with open("../../config/config.yaml", "r") as f:
    config = yaml.safe_load(f)
    
MIN_OVERVIEW_WORDS = config["data_preprocessing"]["min_overview_words"] #6
MAX_OVERVIEW_WORDS = config["data_preprocessing"]["max_overview_words"] #169

tmdb_df = pd.read_csv("../../data/raw/TMDB_movie_dataset_v11.csv", usecols=['title', 'vote_average', 'vote_count', 'popularity', 'release_date', 'status', 'runtime', 'adult', "overview", 'genres', 'production_companies', 'production_countries', 'keywords'])
print(f"TMDB shape: {tmdb_df.shape}")
tmdb_df.head()

TMDB shape: (1367573, 13)


,title,vote_average,vote_count,status,release_date,runtime,adult,overview,popularity,genres,production_companies,production_countries,keywords
0,Inception,8.364,34495,Released,2010-07-15,148,False,"Cobb, a skilled thief who commits corporate es...",83.952,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","rescue, mission, dream, airplane, paris, franc..."
1,Interstellar,8.417,32571,Released,2014-11-05,169,False,The adventures of a group of explorers who mak...,140.241,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America","rescue, future, spacecraft, race against time,..."
2,The Dark Knight,8.512,30619,Released,2008-07-16,152,False,Batman raises the stakes in his war on crime. ...,130.643,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","joker, sadism, chaos, secret identity, crime f..."
3,Avatar,7.573,29815,Released,2009-12-15,162,False,"In the 22nd century, a paraplegic Marine is di...",79.932,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","future, society, culture clash, space travel, ..."
4,The Avengers,7.710,29166,Released,2012-04-25,143,False,When an unexpected enemy emerges and threatens...,98.082,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"new york city, superhero, shield, based on com..."


In [2]:
letterbox = []
for line in open("../../data/raw/full_dump.jsonl", "r"):
    letterbox.append(json.loads(line))

letterbox_df = pd.DataFrame(letterbox)
letterbox_df.drop(['url', 'reviews', 'poster_url', 'rating'], axis=1, inplace=True)
letterbox_df.drop_duplicates(subset=['title'], inplace=True, ignore_index=True)
letterbox_df

,title,year,directors,genres,cast,synopsis
0,Come and See,1985,[Elem Klimov],"[War, Drama]","[Aleksei Kravchenko, Olga Mironova, Liubomiras...",The invasion of a village in Byelorussia by Ge...
1,Seven Samurai,1954,[Akira Kurosawa],"[Action, Drama]","[Toshirō Mifune, Takashi Shimura, Yoshio Inaba...",A samurai answers a village's request for prot...
2,High and Low,1963,[Akira Kurosawa],"[Mystery, Thriller, Crime, Drama]","[Toshirō Mifune, Tatsuya Nakadai, Kyōko Kagawa...",In the midst of an attempt to take over his co...
3,Harakiri,1962,[Masaki Kobayashi],"[History, Drama, Action]","[Tatsuya Nakadai, Akira Ishihama, Shima Iwashi...",Down-on-his-luck veteran Tsugumo Hanshirō ente...
4,12 Angry Men,1957,[Sidney Lumet],[Drama],"[Martin Balsam, John Fiedler, Lee J. Cobb, E.G...",The defense and the prosecution have rested an...
...,...,...,...,...,...,...
694511,Road Wars: Max Fury,2024,[Mark Atkins],"[Science Fiction, Adventure, Action]","[Chandni Shah, Preet Kaur, Lindsey Marie Wilso...","In a post-apocalyptic wasteland, two sisters l..."
694512,El Fil Fel Mandil,2011,[Ahmed ElBadry],[Comedy],"[Talaat Zakaria, Nana, Mohamed El Sobky, Ramy ...",An unassuming fruit seller is assigned a cruci...
694513,Wonderwell,2023,[Vlad Marsavin],"[Family, Fantasy, Drama]","[Kiera Milward, Nell Tiger Free, Carrie Fisher...","In an idyllic Italian village, a young America..."
694514,The Changeover,2017,"[Stuart McKenzie, Miranda Harcourt]","[Fantasy, Horror, Thriller]","[Timothy Spall, Melanie Lynskey, Lucy Lawless,...","Laura Chant, 16, lives with her mother and fou..."


In [3]:
# Нормализация названий для матчинга
def normalize_title(title):
    if pd.isna(title):
        return ""
    title = str(title).lower().strip()
    title = title.replace("the ", "").replace("a ", "").replace("an ", "")
    title = ''.join(c for c in title if c.isalnum() or c.isspace())
    return ' '.join(title.split())

tmdb = tmdb_df.copy()
letterbox = letterbox_df.copy()

tmdb['title_norm'] = tmdb['title'].apply(normalize_title)
letterbox['title_norm'] = letterbox['title'].apply(normalize_title)

tmdb['year'] = pd.to_datetime(tmdb['release_date'], errors='coerce').dt.year.astype('Int64')
letterbox['year'] = letterbox['year'].astype('Int64')

# Full outer join на normalized title и year
merged = pd.merge(
    tmdb,
    letterbox,
    how='outer',
    on=['title_norm', 'year'],
    suffixes=('_tmdb', '_lb')
)

# Удаляем дубликаты по title_norm (если нужно, добавь year в subset)
merged = merged.drop_duplicates(subset=['title_norm'], keep='first')

# Убираем временную колонку
merged = merged.drop(columns=['title_norm'], errors='ignore')
merged.reset_index(drop=True, inplace=True)
# Сохраняем
merged.to_csv('../../data/raw/union_tmdb_letterboxd.csv', index=False)

print(f"Merged shape: {merged.shape}")
merged = merged[["title_tmdb", "title_lb", "status", "release_date", "year", "vote_average", "vote_count", "popularity", "runtime", "adult", "overview", "synopsis", "genres_tmdb", "genres_lb", "directors", "cast", "production_countries", "production_companies", "keywords"]]
print(f"Merged shape: {merged.shape}")

print(merged.info())
merged


Merged shape: (1168081, 19)
Merged shape: (1168081, 19)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1168081 entries, 0 to 1168080
Data columns (total 19 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   title_tmdb            1075443 non-null  object 
 1   title_lb              635226 non-null   object 
 2   status                1075443 non-null  object 
 3   release_date          888534 non-null   object 
 4   year                  975784 non-null   Int64  
 5   vote_average          1075443 non-null  float64
 6   vote_count            1075443 non-null  float64
 7   popularity            1075443 non-null  float64
 8   runtime               1075443 non-null  float64
 9   adult                 1075443 non-null  object 
 10  overview              818172 non-null   object 
 11  synopsis              535340 non-null   object 
 12  genres_tmdb           612946 non-null   object 
 13  genres_lb             476472 no

,title_tmdb,title_lb,status,release_date,year,vote_average,vote_count,popularity,runtime,adult,overview,synopsis,genres_tmdb,genres_lb,directors,cast,production_countries,production_companies,keywords
0,,NaN,Released,1921-09-01,1921,0.0,0.0,0.6000,6.0,False,A look at an exhibition in Reval (now known as...,NaN,Documentary,NaN,NaN,NaN,Soviet Union,NaN,NaN
1,0,NaN,Released,1977-01-01,1977,0.0,0.0,0.6000,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,France,NaN,NaN
2,0 & 1,0 & 1,Released,2002-02-14,2002,0.0,0.0,1.4000,74.0,False,"A girl called Zero stands in a crowded street,...","A girl called Zero stands in a crowded street,...","Drama, Crime","[Drama, Crime]",[Kei Nakata],"[Eri Ishikawa, Takayuki Katô, Joe Yamanaka, Is...",Japan,NaN,NaN
3,0 and 26,0 and 26,Released,2016-12-23,2016,8.5,2.0,0.6000,43.0,False,The Tampa Bay Buccaneers losing their first ev...,The Tampa Bay Buccaneers losing their first ev...,Documentary,[Documentary],NaN,"[Ron Wolf, Steve Spurrier, J.K. McKay, Rich Mc...",NaN,NFL Films,NaN
4,0% Down,0% Down,Released,2008-01-01,2008,0.0,0.0,0.6000,6.0,False,Meckseper's parodic high-octane video 0% Down ...,Meckseper's parodic high-octane video 0% Down ...,NaN,NaN,[Josephine Meckseper],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168076,𝒀𝑬𝑳𝑳𝑶𝑾,NaN,Released,NaN,<NA>,0.0,0.0,0.6000,10.0,False,When an alleged business meeting turns into an...,NaN,"Drama, Thriller",NaN,NaN,NaN,NaN,One Way Mirror,NaN
1168077,{𝕮𝖆𝖑𝖑 𝖀𝖘} How do I talk to someone through BLO...,NaN,Released,NaN,<NA>,0.0,0.0,0.0000,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1168078,𝗦𝗮𝗻𝘁𝗮 𝗔𝗻𝗮: 𝗧𝗵𝗲 𝗪𝗼𝗺𝗯 𝗼𝗳 𝗠𝗮𝗻𝗶𝗹𝗮,𝗦𝗮𝗻𝘁𝗮 𝗔𝗻𝗮: 𝗧𝗵𝗲 𝗪𝗼𝗺𝗯 𝗼𝗳 𝗠𝗮𝗻𝗶𝗹𝗮,Released,2023-11-21,2023,0.0,0.0,0.0000,36.0,False,"The film is about inspiration, reminding the p...","The film is about inspiration, reminding the p...",Documentary,[Documentary],[Regie Mae Adelantar],"[Lou Veloso, Boyet Magale, Sylvia Lichauco De ...",Philippines,The Archivist,"philippines, manila philippines, cultural iden..."
1168079,𝗧𝗵𝗲 𝗗𝗶𝘀𝗮𝗽𝗽𝗲𝗮𝗿𝗮𝗻𝗰𝗲 𝗼𝗳 𝗦𝗮𝗱𝗶𝗲 𝗙𝗶𝗻𝗹𝗼𝗰𝗵,NaN,Released,2025-09-07,2025,0.0,0.0,0.0357,0.0,False,Sadie Finloch disappeared in 1973. What happened?,NaN,NaN,NaN,NaN,NaN,United Kingdom,NaN,NaN


In [4]:
#working with title
merged['title'] = merged['title_tmdb'].combine_first(merged['title_lb'])
merged.drop(columns=['title_tmdb', 'title_lb'], inplace=True)

In [5]:
#working with release_date
print(merged['status'].value_counts())
merged.drop(merged[merged['status'].isin(["Canceled", "Rumored", "Planned"])].index, inplace=True)
print(merged['status'].value_counts())

merged['release_info'] = (
    merged['release_date']
    .combine_first(merged['year'])
    .combine_first(merged['status'])
)
merged.drop(columns=['release_date', 'year', 'status'], inplace=True)


status
Released           1045665
In Production        13880
Post Production       8472
Planned               6743
Rumored                452
Canceled               231
Name: count, dtype: int64
status
Released           1045665
In Production        13880
Post Production       8472
Name: count, dtype: int64


In [6]:
merged["overview_new"] = merged["overview"].combine_first(merged["synopsis"])
merged.drop(columns=['overview', 'synopsis'], inplace=True)

In [7]:
merged["genres"] = merged['genres_tmdb'].combine_first(merged['genres_lb'])
merged.drop(columns=['genres_tmdb', 'genres_lb'], inplace=True)

In [8]:
#TODO убрать все ненужные данные (adult true) подумать про runtime 0, release неясный 
#TODO и затем попробовать объединить рейтинг
merged = merged[['title', 'release_info', 'overview_new', 'vote_average', 'vote_count', 'popularity', 'runtime', 'genres', 'adult','directors', 'cast', 'production_countries', 'production_companies', 'keywords']]
merged.rename(columns={'overview_new':'overview'}, inplace=True)
merged.reset_index(drop=True, inplace=True)
print(merged.info())
merged

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1160655 entries, 0 to 1160654
Data columns (total 14 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   title                 1160655 non-null  object 
 1   release_info          1155267 non-null  object 
 2   overview              905847 non-null   object 
 3   vote_average          1068017 non-null  float64
 4   vote_count            1068017 non-null  float64
 5   popularity            1068017 non-null  float64
 6   runtime               1068017 non-null  float64
 7   genres                700595 non-null   object 
 8   adult                 1068017 non-null  object 
 9   directors             536533 non-null   object 
 10  cast                  439421 non-null   object 
 11  production_countries  571588 non-null   object 
 12  production_companies  474419 non-null   object 
 13  keywords              270570 non-null   object 
dtypes: float64(4), object(10)
memory u

,title,release_info,overview,vote_average,vote_count,popularity,runtime,genres,adult,directors,cast,production_countries,production_companies,keywords
0,,1921-09-01,A look at an exhibition in Reval (now known as...,0.0,0.0,0.6000,6.0,Documentary,False,NaN,NaN,Soviet Union,NaN,NaN
1,0,1977-01-01,NaN,0.0,0.0,0.6000,0.0,NaN,False,NaN,NaN,France,NaN,NaN
2,0 & 1,2002-02-14,"A girl called Zero stands in a crowded street,...",0.0,0.0,1.4000,74.0,"Drama, Crime",False,[Kei Nakata],"[Eri Ishikawa, Takayuki Katô, Joe Yamanaka, Is...",Japan,NaN,NaN
3,0 and 26,2016-12-23,The Tampa Bay Buccaneers losing their first ev...,8.5,2.0,0.6000,43.0,Documentary,False,NaN,"[Ron Wolf, Steve Spurrier, J.K. McKay, Rich Mc...",NaN,NFL Films,NaN
4,0% Down,2008-01-01,Meckseper's parodic high-octane video 0% Down ...,0.0,0.0,0.6000,6.0,NaN,False,[Josephine Meckseper],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1160650,𝒀𝑬𝑳𝑳𝑶𝑾,Released,When an alleged business meeting turns into an...,0.0,0.0,0.6000,10.0,"Drama, Thriller",False,NaN,NaN,NaN,One Way Mirror,NaN
1160651,{𝕮𝖆𝖑𝖑 𝖀𝖘} How do I talk to someone through BLO...,Released,NaN,0.0,0.0,0.0000,0.0,NaN,False,NaN,NaN,NaN,NaN,NaN
1160652,𝗦𝗮𝗻𝘁𝗮 𝗔𝗻𝗮: 𝗧𝗵𝗲 𝗪𝗼𝗺𝗯 𝗼𝗳 𝗠𝗮𝗻𝗶𝗹𝗮,2023-11-21,"The film is about inspiration, reminding the p...",0.0,0.0,0.0000,36.0,Documentary,False,[Regie Mae Adelantar],"[Lou Veloso, Boyet Magale, Sylvia Lichauco De ...",Philippines,The Archivist,"philippines, manila philippines, cultural iden..."
1160653,𝗧𝗵𝗲 𝗗𝗶𝘀𝗮𝗽𝗽𝗲𝗮𝗿𝗮𝗻𝗰𝗲 𝗼𝗳 𝗦𝗮𝗱𝗶𝗲 𝗙𝗶𝗻𝗹𝗼𝗰𝗵,2025-09-07,Sadie Finloch disappeared in 1973. What happened?,0.0,0.0,0.0357,0.0,NaN,False,NaN,NaN,United Kingdom,NaN,NaN


In [9]:
merged.drop(merged[merged['title'].isin([" ", ""])].index, inplace=True)

In [10]:
merged.dropna(subset=['overview'], inplace=True, ignore_index=True)
merged['overview_length'] = merged['overview'].apply(lambda x: len(x.split()))
merged.drop(merged[merged['overview_length'] < MIN_OVERVIEW_WORDS].index, inplace=True)
merged.reset_index(drop=True, inplace=True)

In [11]:
merged.drop(merged[merged['adult'] == True].index, inplace=True)
merged.drop('adult', axis=1, inplace=True)
merged.reset_index(drop=True, inplace=True)
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 773275 entries, 0 to 773274
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   title                 773275 non-null  object 
 1   release_info          770320 non-null  object 
 2   overview              773275 non-null  object 
 3   vote_average          699017 non-null  float64
 4   vote_count            699017 non-null  float64
 5   popularity            699017 non-null  float64
 6   runtime               699017 non-null  float64
 7   genres                553050 non-null  object 
 8   directors             452885 non-null  object 
 9   cast                  373772 non-null  object 
 10  production_countries  398653 non-null  object 
 11  production_companies  310750 non-null  object 
 12  keywords              192738 non-null  object 
 13  overview_length       773275 non-null  int64  
dtypes: float64(4), int64(1), object(9)
memory usage: 82.

In [12]:
merged['overview_length'].describe()

count    773275.000000
mean         47.202890
std          34.568713
min           6.000000
25%          21.000000
50%          37.000000
75%          65.000000
max         237.000000
Name: overview_length, dtype: float64

In [13]:
def truncate_overview(text: str) -> str:
    
    words = text.split()
    if len(words) > MAX_OVERVIEW_WORDS:
        return " ".join(words[:MAX_OVERVIEW_WORDS]) + '...'
    return text

merged['overview'] = merged['overview'].apply(truncate_overview)

In [14]:
merged['overview_length'] = merged['overview'].apply(lambda x: len(x.split()))
merged['overview_length'].describe()

count    773275.000000
mean         47.187261
std          34.510755
min           6.000000
25%          21.000000
50%          37.000000
75%          65.000000
max         169.000000
Name: overview_length, dtype: float64

In [19]:
merged.drop('overview_length', axis=1, inplace=True)
merged.info()
merged

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 773275 entries, 0 to 773274
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   title                 773275 non-null  object 
 1   release_info          770320 non-null  object 
 2   overview              773275 non-null  object 
 3   vote_average          699017 non-null  float64
 4   vote_count            699017 non-null  float64
 5   popularity            699017 non-null  float64
 6   runtime               699017 non-null  float64
 7   genres                553050 non-null  object 
 8   directors             452885 non-null  object 
 9   cast                  373772 non-null  object 
 10  production_countries  398653 non-null  object 
 11  production_companies  310750 non-null  object 
 12  keywords              192738 non-null  object 
dtypes: float64(4), object(9)
memory usage: 76.7+ MB


,title,release_info,overview,vote_average,vote_count,popularity,runtime,genres,directors,cast,production_countries,production_companies,keywords
0,0 & 1,2002-02-14,"A girl called Zero stands in a crowded street,...",0.0,0.0,1.4000,74.0,"Drama, Crime",[Kei Nakata],"[Eri Ishikawa, Takayuki Katô, Joe Yamanaka, Is...",Japan,NaN,NaN
1,0 and 26,2016-12-23,The Tampa Bay Buccaneers losing their first ev...,8.5,2.0,0.6000,43.0,Documentary,NaN,"[Ron Wolf, Steve Spurrier, J.K. McKay, Rich Mc...",NaN,NFL Films,NaN
2,0% Down,2008-01-01,Meckseper's parodic high-octane video 0% Down ...,0.0,0.0,0.6000,6.0,NaN,[Josephine Meckseper],NaN,NaN,NaN,NaN
3,0 Feet Away,2014-12-15,"Looking for a good time, Chase stays home one ...",0.0,0.0,0.6000,10.0,NaN,NaN,NaN,NaN,NaN,NaN
4,0 Lux,Released,A man clearing out his recently deceased fathe...,0.0,0.0,0.0000,7.0,Horror,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
773270,𝐓𝐡𝐞 𝐕𝐢𝐠𝐢𝐥𝐚𝐧𝐭 𝐕𝐢𝐥𝐥𝐚𝐠𝐞𝐫𝐬,Released,This is the story of three Filipino rural comm...,0.0,0.0,0.0000,42.0,NaN,NaN,NaN,NaN,NaN,NaN
773271,𝒀𝑬𝑳𝑳𝑶𝑾,Released,When an alleged business meeting turns into an...,0.0,0.0,0.6000,10.0,"Drama, Thriller",NaN,NaN,NaN,One Way Mirror,NaN
773272,𝗦𝗮𝗻𝘁𝗮 𝗔𝗻𝗮: 𝗧𝗵𝗲 𝗪𝗼𝗺𝗯 𝗼𝗳 𝗠𝗮𝗻𝗶𝗹𝗮,2023-11-21,"The film is about inspiration, reminding the p...",0.0,0.0,0.0000,36.0,Documentary,[Regie Mae Adelantar],"[Lou Veloso, Boyet Magale, Sylvia Lichauco De ...",Philippines,The Archivist,"philippines, manila philippines, cultural iden..."
773273,𝗧𝗵𝗲 𝗗𝗶𝘀𝗮𝗽𝗽𝗲𝗮𝗿𝗮𝗻𝗰𝗲 𝗼𝗳 𝗦𝗮𝗱𝗶𝗲 𝗙𝗶𝗻𝗹𝗼𝗰𝗵,2025-09-07,Sadie Finloch disappeared in 1973. What happened?,0.0,0.0,0.0357,0.0,NaN,NaN,NaN,United Kingdom,NaN,NaN


In [2]:
!pwd

/home/vanishhhed/Desktop/codics/PythonCodics/ML/RAG/src/dataset


In [ ]:
import requests
url = "https://huggingface.co/datasets/pkchwy/letterboxd-all-movie-data/resolve/main/full_dump.jsonl"
response = requests.get(url, stream=True)
with open('full_dump.jsonl', 'wb') as f:
    f.write(response.content)
print("Скачано в full_dump.jsonl")

In [6]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("pkchwy/letterboxd-all-movie-data", data_files="full_dump.jsonl",cache_dir="./")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
import pandas as pd

# Login using e.g. `huggingface-cli login` to access this dataset
df = pd.read_json("hf://datasets/pkchwy/letterboxd-all-movie-data/full_dump.jsonl", lines=True)

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("asaniczka/tmdb-movies-dataset-2023-930k-movies")

print("Path to dataset files:", path)

KaggleApiHTTPError: 404 Client Error.

Resource not found at URL: https://www.kaggle.com/datasets/asaniczka/tmdb-movies-dataset-2023-930k-movies/versions/847
The server reported the following issues: Dataset not found
Please make sure you specified the correct resource identifiers.